<a href="https://colab.research.google.com/github/gihuni99/Basic_MLDL/blob/main/Ch04_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch04-1 Logistic Regression

럭키백의 구성품은 모르는 상태로 판매를 기획. 구성품은 모두 생선이고, 럭키백에 포함된 생선의 확률을 알려주는 방향으로 판매

### 럭키백의 확률

럭키백에 들어갈 수 있는 생선은 7가지.

럭키백에 들어간 생선의 크기, 무게 등이 주어졌을 때, 7가지의 생선에 대한 확률을 출력.

K-NN 알고리즘을 사용하여 이웃하는 클래스의 비율을 확률값으로 출력해보자

_**1. 데이터 준비하기**_

모델 training에 사용할 데이터를 만들어 보자. Pandas를 활용하여 CSV파일을 불러오겠다.

read_csv()함수를 통해 csv파일을 dataframe으로 변환하고, head() method를 통해 처음 5개 행을 출력

아래의 왼쪽 숫자는 row number(pandas의 index), 위의 Species, Weight 등은 column name이다. pandas는 CSV파일의 첫 행을 자동으로 인식해 Column name으로 만들어 준다.

+) **dataframe**이란 pandas에서 제공하는 2차원 표 형식의 주요 데이터 구조. 통계와 그래프를 위한 method를 제공하고, numpy로의 상호 변환이 쉽다.

In [ ]:
import pandas as pd
fish=pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() # head()를 통해 처음 5개 행을 출력

이제 어떤 종류의 생선들이 있는지 'Species' column에서 값을 추출해보자. pandas의 **unique()**함수를 사용하면 간단하다.

In [ ]:
print(pd.unique(fish['Species']))

위 출력을 통해 총 7종의 생선이 있다는 것을 알 수 있다. 'Species' Column은 target으로 설정하고, 나머지 5개의 column을 input data로 활용하겠다.

dataframe에서 여러 열을 선택하면 새로운 dataframe이 반환된다. 이를 **to_numpy() method로 dataframe에서 numpy array로 변환하고, fish_input에 저장한다.

In [ ]:
fish_input=fish[['Weight','Length','Diagonal','Height','Width']].to_numpy() #input data는 2차원 배열이 되어야 한다.

아래 결과를 통해 5개의 feature값이 잘 저장된 것을 볼 수 있다. 위의 CSV파일의 구성과 비교하면 동일하다.

In [ ]:
print(fish_input[:5])

이제 fish 데이터의 'Species' column을 target data로 변환해보자.

이때, 'Species' column을 선택할 때, fish[['Species']]와 같이 2개의 괄호를 사용하지 않도록 주의해야 한다. 이렇게 코딩한다면 target data가 2차원 배열이 된다.

In [ ]:
fish_target=fish['Species'].to_numpy() #target data는 1차원

In [ ]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(fish_input,fish_target,random_state=42) #train set과 test set으로 분할

위의 train_test_split을 통해 train set과 test set을 분할하고, StandardScaler() 클래스를 통해 data preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()## StandardScaler 클래스를 통해 train set과 test set을 preprocessing
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input) #train set 통계 값으로 test set 변환

_**2. K-NN Classifier의 확률 예측**_

scikit-learn의 KNeighborsClassifier 클래스 객체를 만든 후 training하여 결과 확인(k=3으로 지정)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled,train_target)
print(kn.score(train_scaled,train_target))
print(kn.score(test_scaled,test_target))

우선 클래스의 확률을 배우는 것이 목적이므로 score에 대한 생각은 배제.

**Multi-class Classification**: target data에 2개 이상의 class가 포함된 것을 Multi-class Classification이라고 한다.

이전의 Binary Classification 예시에서는 1, 0으로 지정하여 target data를 생성하였다. Muti-class Classification에서도 target값을 숫자로 변환하여 입력할 수 있지만, scikit-learn에서는 문자열로 된 target값을 그대로 사용 가능하다.

**주의)** target값을 그대로 scikit-learn모델에 전달하면 순서가 자동으로 알파벳 순서대로 나열된다. 따라서, **pd.unique(fish['Species'])로 출력했던 순서와 다르다.

KNeighborsClassifier에서 정렬된 target값은 'classes_' attribution에 저장되어 있다.

In [ ]:
print(kn.classes_)

위 출력값을 통해 첫번째 클래스는 'Bream', 두번째 클래스는 'Parkki' 순으로 나오는 것을 볼 수 있다.

이제 **predict()** method를 통해 test set의 첫 5개의 sample에 대한 class를 예측해보자.

In [ ]:
print(kn.predict(test_scaled[:5]))

Scikit-Learn의 classification 모델은 **predict_proba()** method로 클래스별 확률값을 반환한다. 이를 통해 처음 5개의 sample에 대한 확률을 출력해보자.

numpy의 round()함수는 기본적으로 소수점 첫째자리에서 반올림을 한다.(decimals 변수를 통해 유지할 소수점 아래 자리수를 지정할 수 있다)

In [ ]:
import numpy as np
proba=kn.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=4))

위 결과에서 predict_proba() method의 출력 순서는 앞서 보았던 'classes_'attribution과 같다.

따라서 첫번째 열이 'Bream' 클래스에 대한 확률, 두번째 열이 'Parkki' 클래스에 대한 확률이다.

위 결과에 따르면 4, 5번째 Sample이 'Perch'일 확률은 66.7%, 'Roach'일 확률은 33.3%이다. 이에 대한 결과를 검증해보자

4번째 sample에 대한 결과를 k-neighbor확인을 통해 검증해보겠다.

In [ ]:
distances,indexes=kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

위 결과를 통해, 3개의 K중 2개는 'Perch', 1개는 'Roach'임을 알 수 있고, 확률이 정확하게 나온 것을 검증할 수 있다.

+ Tip) kneighbors() method의 입력은 2차원 배열이어야 한다. 이를 위해 numpy의 slicing을 사용했다. slicing operator는 하나의 sample을 선택해도 항상 2차원 배열이 만들어진다.

위 방법대로 한다면 확률이 0,33,66,100의 4가지 정도로만 나오게 된다. 따라서 더 좋은 방법은 다음 실습에서 진행하고자 한다.

### Logistic Regression

**Logistic Regression**은 이름과 달리 classification model이다. 해당 모델은 Linear Regression과 동일하게 Linear equation을 학습한다.

Ex) z=a x (Weight) + b x (Length) + c x (Diagonal) + d x (Height) + e x (Width) + f

위 식에서 a, b, c, d, e는 weight(가중치)이다. Feature는 늘어났지만 Multiple Regression을 위한 Linear equation이다. z는 어떤 값이든 가능하지만 확률이 되기 위해서는 0~1사이의 값이 되어야 한다. 따라서 **Sigmoid Function(Logistic Function)**을 사용한다.

$phi=1/(1+e^-z)$ (sigmoid 함수)

위 식을 사용하면 값이 0~1사이의 값으로 정규화 된다. 코드로 작성하면 아래와 같다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
z=np.arange(-5,5,0.1)
phi=1/(1+np.exp(-z))
plt.plot(z,phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

_**Logistic Regression으로 binary classification수행**_

Sigmoid Function을 사용하면 함수의 출력이 0.5보다 크면 '1' 클래스, 작으면 '0'클래스로 분류할 수 있다.

Ex)

numpy array는 True, False값을 전달하여 행을 선택할 수 있다. 이를 **Boolean indexing**이라고 한다.

In [ ]:
char_arr=np.array(['A','B','C','D','E'])#해당 배열에서 'A'와 'C'만을 골라내기 위해서 첫번째와 세번째 element만 True인 배열을 전달한다
print(char_arr[[True,False,True,False,False]])

위와 같은 방식으로 Train set에서 Bream과 Smelt의 행만 골라보자.

비교연산자를 통해 train_target이 'Bream' 또는 'Smelt'인 element를 찾고 True, False를 갖는 배열을 반환한다.

In [ ]:
bream_smelt_indexes=(train_target=='Bream')|(train_target=='Smelt') #or 연산자를 통해 'Bream'과 'Smelt'를 갖는 index에 모두 True를 반환하도록
train_bream_smelt=train_scaled[bream_smelt_indexes]
target_bream_smelt=train_target[bream_smelt_indexes]

bream_smelt_indexes 배열에는 'Bream'과 'Smelt'를 제외한 index에는 False가 들어가 있다. 이를 통해 boolean indexing을 적용하여 'Bream'과 'Smelt'데이터만을 추릴 수 있다.

이제 해당 데이터로 Logistic Regression 모델을 훈련시켜보자. **LogisticRegression** 클래스는 sklearn.linear_model 패키지 아래에 있다.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(train_bream_smelt,target_bream_smelt)

아래 결과를 보면 2번째 sample을 제외하고 모두 'Bream'으로 예측한 것을 볼 수 있다. KNeighborsClassifier과 마찬가지로 예측확률은 **predict_proba()** method에서 제공한다.

In [ ]:
print(lr.predict(train_bream_smelt[:5]))

In [ ]:
print(lr.predict_proba(train_bream_smelt[:5]))

위 **predict_proba**의 출력 결과를 보면 각 sample마다 2개의 확률이 출력된 것을 볼 수 있다. 첫번째 열이 0(음성 클래스)에 대한 확률이고, 두번째 열이 1(양성 클래스)에 대한 확률이다.

이제 Bream과 Smelt 중 어떤 것이 양성 클래스인지 확인해보자.(K-NN classifier에서와 같이, scikit-learn은 target값을 알파벳 순으로 정렬한다. 이는 **classes_** attribution에서 확인 가능하다.

In [ ]:
print(lr.classes_)

위 결과를 통해 Smelt가 양성 클래스임을 알 수 있다. Logistic Regression으로 binary classification을 완료하였다.

이제 Logistic Regression 모델이 학습한 weight를 확인해보자.

In [ ]:
print(lr.coef_,lr.intercept_) #coef_와 intercept는 각각 weight와 bias를 저장하는 attribution이다.

z= -0.404 x (Weight) - 0.576 x (Length) - 0.663 x (Diagonal) - 1.013 x (Height) - 0.732 x (Width) - 2.161 의 식이 나온다. 이는 Linear regression과 매우 유사한 형태.

이제 z값을 구해보자. **LogisticRegression** 클래스는 **decision_function()** method를 통해 z값을 출력할 수 있다.

In [ ]:
decisions=lr.decision_function(train_bream_smelt[:5])
print(decisions)

**decision_function**을 통해 얻은 z값을 sigmoid function에 넣으면 확률값을 얻을 수 있다.

python의 **scipy(사이파이)** library에는 sigmoid function이 있다. 바로 **expit()**이다. 이전에 구했던 np.exp()를 사용해 구하는 것보다 편리하다.

In [ ]:
from scipy.special import expit
print(expit(decisions))

위 출력값을 보면 **predict_proba()** method를 통한 출력의 2번째 column과 값이 동일하다. 따라서 **decision_function()** method는 양성 클래스(1)에 대한 z값을 반환한다는 것을 알 수 있다.

_**Logistic Regression으로 Multiple Classification 수행하기**_

**LogisticRegression** 클래스는 반복적인 알고리즘을 사용한다.(max_iter 변수를 통해 반복횟수를 지정, 기본값은 100) 충분한 반복을 위해 max_iter=1000으로 설정

또한 기본적으로 Ridge Regression과 같이 weight의 제곱을 Regularization한다.(L2 Regularization) Ridge Regression에서는 alpha로 Regularization의 강도를 조절했지만, LogisticRegression에서는 **C** parameter를 통해 조절한다. alpha와 반대로 **C**값이 작을 수록 Regularization의 강도가 세진다.(기본값은 1) 예시에서는 Regularization의 강도를 낮추기 위해 C=20으로 설정

In [ ]:
lr=LogisticRegression(C=20,max_iter=1000) #Regularization의 강도는 C=20
lr.fit(train_scaled,train_target) #train_scaled는 5개의 feature로 이루어져 있고, 각 feature값에 따라 target을 예측하도록 학습
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

위 결과를 통해 학습이 잘 이루어졌음을 확인할 수 있다.

아래 예측값을 확인해보면 test set에서 처음 5개의 sample에 대한 Class를 예측하였다.

In [ ]:
print(lr.predict(test_scaled[:5]))

다음으로 predict_proba를 통해 test set에서 첫 5개의 sample에 대한 예측 확률을 출력해보았다.

In [ ]:
proba=lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3)) #각 확률값은 소수 셋째자리까지 반올림

위 결과를 보면 5개의 sample에 대한 예측이므로 행이 5개이고, 7개의 class(target)에 대한 예측이므로 열이 7개이다.

아래의 classes_를 출력해보면 세번째 class가 'Perch'이므로 1번째 sample은 가장 높은 확률을 갖는 'Perch'로 class를 예측한 것을 확인할 수 있다.

In [ ]:
print(lr.classes_)

그렇다면 multiple classification의 경우 linear equation은 어떻게 이루어지는지 확인해보자.

아래 결과를 보면 coef_의 크기는 (7,5), intercept_의 크기는 (7,)임을 알 수 있다.

5개의 feature를 사용하므로 coef_의 열은 5개이고, coef_와 inercept_의 행이 모두 7개임을 알 수 있다. 즉, 7개의 z값을 계산한다는 것이다.

(이전의 binary classification에서는 coef_는 (1,5), intercept_는 (1,)의 크기를 갖고, 하나의 z값을 계산하여 z값에 대한 sigmoid function값을 계산한 후 각 sample의 양성 클래스에 대한 확률을 구했다.)

multiple classification에서는 클래스마다의 z값을 하나씩 계산한다. 따라서 높은 z값을 출력하는 클래스가 예측값이 되는 것이다. 또한 binary classification에서 sigmoid function을 사용한 것과는 다르게, **softmax function**을 사용하여 7개의 z값을 확률값으로 변환한다.

In [ ]:
print(lr.coef_.shape,lr.intercept_.shape)

**Softmax Function**

7개(각 클래스 별)의 z값은 z1~z7이라고 할 때, e_sum=$e^{z1} + e^{z2}+ e^{z3}+ e^{z4}+ e^{z5}+ e^{z6}+ e^{z7}$ 이라고 하자.

각 1개의 sample에 대한 class의 확률을 각각 s1~s7이라고 할 때, $s1=e^{z1}$/e_sum, .... , $s7=e^{z7}$/e_sum이다.

s1~s7의 합은 1이다.

이제 **decision_function**을 통해 z1~z7의 값을 구해 softmax function을 통해 확률값으로 변환해보자. test set에 대한 처음 5개의 sample에 대해 값을 구하겠다.

In [ ]:
decision=lr.decision_function(test_scaled[:5]) #decision_function method를 사용하여 5개 sample에 대한 z1~z7 산출
print(np.round(decision,decimals=2))#소수 둘째자리까지 반올림

scipy(사이파이)는 softmax함수를 제공(scipy.special 아래에 있다.)

decision배열은 softmax()함수에 전달하였다. 'axis' parameter는 softmax를 계산할 축을 지정한다. 여기서 axis=1로 지정하여 각 행(sample)에 대해 softmax를 계산하도록 하였다.(만약 지정하지 않으면 배열 전체에 대한 softmax 계산)

In [ ]:
from scipy.special import softmax
proba=softmax(decision,axis=1)
print(np.round(proba,decimals=3))

아래 결과를 보면 predict값과 확률값이 일치하는 것을 확인할 수 있다.

In [ ]:
print(lr.classes_)
print(lr.predict(test_scaled[:5]))

### 추가공부

**Scikit-Learn**
* **LogisticRegression**: Linear Classification 알고리즘인 Logistic Regression을 위한 클래스. **solver** parameter에서 사용할 알고리즘 선택가능(기본값은 1bfgs, 'sag',saga'지정 가능), **penalty** parameter에서 L2 또는 L1 Regularization을 선택할 수 있다(기본값은 L2), **C** parameter에서 Regularization의 강도를 제어(기본값은 1, 값이 작을수록 regularization강도 세진다)

* **predict_proba()**: 예측 확률을 반환한다. binary classification의 경우 각 sample에 대한 양성 클래스와 음성 클래스에 대한 확률을 반환, multiple classification의 경우 각 sample마다 모든 클래스에 대한 확률을 반환

* **decision_function**: 모델이 학습한 linear equation의 출력(z)를 반환한다. binary classification의 경우 양성 클래스에 대한 확률을 반환(z값이 0보다 크면 양성 클래스, 작거나 같으면 음성클래스), multiple classification의 경우 각 클래스마다 linear equation을 계산(가장 큰 값(z)을 갖는 클래스가 예측 클래스)